# Projet Fraude 



In [1]:
#importer vos lib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import DistanceMetric
from time import perf_counter 
import warnings
warnings.simplefilter(action='ignore')

### Importer les dataset

In [2]:
#target --> class : 1 Fraud | 0 ok 

FD = pd.read_csv("./data/fraud_data.csv")
FD.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [3]:
IPA = pd.read_csv("./data/ip_address.csv")
IPA.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


### Créer une nouvelle colonne `country` en effectuant une jointure entre les deux dataset 

In [ ]:
def ip_to_country(ip):
    tmp = IPA[(IPA['lower_bound_ip_address']<ip) & (IPA['upper_bound_ip_address']>=ip)]['country']
    if len(tmp):
        return tmp.tolist()[0]
    else:
        return 'NULL'
    
    
    
FD['country'] = FD.apply(lambda row: ip_to_country(row['ip_address']),axis=1)
FD.head()

### Créer une nouvelle variable `time_diff` qui sera la différence entre les variables `purchase_time` et `signup_time`

In [ ]:
from datetime import datetime
purchase = FD['purchase_time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
signup = FD['signup_time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

FD['time_diff'] = purchase - signup
FD['time_diff'] = FD['time_diff'].apply(lambda x: x.days)
FD.head()

### Regroupper les `user_id` par `device_id`  afin d'avoir le nombre de device par client  

In [ ]:
device_num = FD[['user_id', 'device_id']].groupby('device_id').count().reset_index()
device_num = device_num.rename(columns={'user_id': 'device_num'})
data = FD.merge(device_num, how='left', on='device_id')
data.head() 

### Faire de même pour les `ip_address` 

In [ ]:
ip_num = FD[['user_id', 'ip_address']].groupby('ip_address').count().reset_index()
ip_num = ip_num.rename(columns={'user_id': 'ip_num'})
data = data.merge(ip_num, how='left', on='ip_address')
data.head()

### Traiter les variables `signup_time` & `purchase_time` afin de les différentier suivant la semaine et le week end  

In [ ]:
from datetime import date 
data.signup_time = pd.to_datetime(data.signup_time)
data.purchase_time = pd.to_datetime(data.purchase_time)
data['signup_week'] = data.signup_time.apply(lambda x: date.isoweekday(x))
data['signup_day'] = data.signup_time.apply(lambda x: x.day)


data['purchase_week'] = data.purchase_time.apply(lambda x: date.isoweekday(x))
data['purchase_day'] = data.purchase_time.apply(lambda x: x.day)
data.head()

### Définir vos variables de travail (X,y)

### Afficher les informations du dataset afin de vizualiser les variables numeriques et categoriques

In [ ]:
data.info()

### Traiter les variables catégoriques

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelEncoder()
X['source'] = lb.fit_transform(X['source'])
X['sex'] = lb.fit_transform(X['sex'])
X['browser'] = lb.fit_transform(X['browser'])
X['country'] = lb.fit_transform(X['country'])
X.head()

### Spliter vos variables 

In [ ]:

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    X_train.head()
    y_train.head()

### Instancier un `RandomForestClassifier` et l'entrainer sur vos données 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators = 20, criterion = 'entropy', max_depth = 50 , min_samples_leaf = 3, min_samples_split = 3, oob_score = True )
RFC.fit(X_train, y_train)

### Afficher l'accuracy moyenne pour 10 'fold cross validation'

In [ ]:
from sklearn.metrics import classification_report
prdiction = RCF.predict(X_test)
prediction
print(classification_report(y_test,prediction))

### Afficher les `feature_importances_`

In [ ]:
import pandas as pd
feature_importances = pd.DataFrame(RFC.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

### Vizualiser les sur un graphique 

### Afficher le `Classification report`

### Traçer la courbe AUC 

### Afficher la matrice de confusion 
Qu'en pensez vous ? 

### Faire une selection de certaine variables et spliter vos varibales 

### Entrainer un classifieur `LogisticRegression` sur ces variables 